### Asynchronous Programming 

####Multiprocessing
* Pros

    - Separate memory space
    - Code is usually straightforward
    - Takes advantage of multiple CPUs & cores
    - Avoids GIL limitations for cPython
    - Eliminates most needs for synchronization primitives unless if you use shared memory (instead, it's more of a communication model for IPC)
    - Child processes are interruptible/killable
    - Python multiprocessing module includes useful abstractions with an interface much like threading.Thread
    - A must with cPython for CPU-bound processing

* Cons

    - IPC a little more complicated with more overhead (communication model vs. shared memory/objects)
    Larger memory footprint

####Threading
* Pros

    - Lightweight - low memory footprint
    - Shared memory - makes access to state from another context easier
    - Allows you to easily make responsive UIs
    - cPython C extension modules that properly release the GIL will run in parallel
    - Great option for I/O-bound applications

* Cons

    - cPython - subject to the GIL
    - Not interruptible/killable
    - If not following a command queue/message pump model (using the Queue module), then manual use of synchronization primitives become a necessity (decisions are needed for the granularity of locking)
    - Code is usually harder to understand and to get right - the potential for race conditions increases dramatically


from: http://stackoverflow.com/a/3046201/842420

In [2]:
import multiprocessing

def worker(num):
    """thread worker function"""
    print 'Worker:', num
    return

if __name__ == '__main__':
    jobs = []
    for i in range(5):
        p = multiprocessing.Process(target=worker, args=(i,))
        jobs.append(p)
        p.start()

Worker: 2
Worker: 3
Worker: 4
Worker: 0
Worker: 1
Worker: 0


* multiprocessing

Process([group [, target [, name [, args [, kwargs]]]]])

- target is a callable object that will execute when the process starts
- args is a tuple of positional arguments passed to target
- kwargs is a dictionary of keyword arguments passed to target .
- name is a string that gives a descriptive name to the process. 
- group is unused and is always set to None . Its presence here is simply to make the construction of a Process mimic the creation of a thread in the threading module.



One difference between the threading and multiprocessing examples is the extra protection for __main__ used in the multiprocessing examples. 

Due to the way the new processes are started, the child process needs to be able to import the script containing the target function. Wrapping the main part of the application in a check for __main__ ensures that it is not run recursively in each child as the module is imported. Another approach is to import the target function from a separate script.

An instance p of Process has the following methods:
* p.is_alive()
Returns True if p is still running.
* p.join([timeout])
Waits for process p to terminate. timeout specifies an optional timeout period. A
process can be joined as many times as you wish, but it is an error for a process to try
and join itself.
* p.run()
The method that runs when the process starts. By default, this invokes target that was
passed to the Process constructor. As an alternative, a process can be defined by inher-
iting from Process and reimplementing run() .
* p.start()
Starts the process.This launches the subprocess that represents the process and invokes p.run() in that subprocess.
* p.terminate()
Forcefully terminates the process. If this is invoked, the process p is terminated immedi-
ately without performing any kind of cleanup actions. If the process p created sub-
processes of its own, those processes will turn into zombies. Some care is required when
using this method. If p holds a lock or is involved with interprocess communication,
terminating it might cause a deadlock or corrupted I/O.
A Process instance p also has the following data attributes:
* p.authkey
The process’ authentication key. Unless explicitly set, this is a 32-character string gener-
ated by os.urandom() .The purpose of this key is to provide security for low-level
interprocess communication involving network connections. Such connections only
work if both ends have the same authentication key.
* p.daemon
A Boolean flag that indicates whether or not the process is daemonic. A daemonic
process is automatically terminated when the Python process that created it terminates.
In addition, a daemonic process is prohibited from creating new processes on its own.
The value of p.daemon must be set before a process is started using p.start() .

* p.exitcode
The integer exit code of the process. If the process is still running, this is None . If the
value is negative, a value of –N means the process was terminated by signal N .
* p.name
The name of the process.
* p.pid
The integer process ID of the process.

In [4]:
import multiprocessing

def worker(num):
    """process worker function"""
    print 'Worker:', num
    return



p = multiprocessing.Process(target=worker, args=(10,), name='example_process')
p.start()

Worker: 4


In [5]:
import multiprocessing

def worker2(num):
    """process worker function 2"""
    print 'Worker2:', num
    return



p = multiprocessing.Process(target=worker2, args=(10,), name='example_process')
p.start()

Worker: 10


In [7]:
import multiprocessing
import time

def worker3(num):
    """process worker function 2"""
    while True:
        print 'Worker2:', num
        time.sleep(10)



p1 = multiprocessing.Process(target=worker3, args=(10,), name='continuous_process')
p1.start()

In [9]:
p1.is_alive()

Worker2: 10
Worker2: 10


True

In [10]:
p1.pid

Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10


3473

In [11]:
p1.name

Worker2: 10
Worker2: 10


'continuous_process'

In [12]:
dir(p)

Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10


['_Popen',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_args',
 '_authkey',
 '_bootstrap',
 '_daemonic',
 '_identity',
 '_kwargs',
 '_name',
 '_parent_pid',
 '_popen',
 '_target',
 '_tempdir',
 'authkey',
 'daemon',
 'exitcode',
 'ident',
 'is_alive',
 'join',
 'name',
 'pid',
 'run',
 'start',
 'terminate']

In [13]:
p1.daemon

Worker2: 10
Worker2: 10


False

In [14]:
p1.exitcode # running in background. Returns None

Worker2: 10
Worker2: 10


In [15]:
p.exitcode # previously completed

Worker2: 10
Worker2: 10


0

In [16]:
p1.join(10) # blocks untill 10 seconds

Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10


In [18]:
p.terminate()
p1.terminate()

Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10
Worker2: 10


 ####multiprocessing supports two types of communication channel between processes:

* Queues

    The Queue class is a near clone of Queue.Queue. For example:

    Queues are thread and process safe.


In [19]:
from multiprocessing import Process, Queue

def f(q):
    q.put([42, None, 'hello'])

if __name__ == '__main__':
    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    print q.get()    # prints "[42, None, 'hello']"
    p.join()


[42, None, 'hello']


In [20]:
p.terminate()

* Pipes

    The Pipe() function returns a pair of connection objects connected by a pipe which by default is duplex (two-way). For example:

In [22]:
from multiprocessing import Process, Pipe

def f(conn):
    conn.send([42, None, 'hello'])
    conn.close()

if __name__ == '__main__':
    parent_conn, child_conn = Pipe()
    p = Process(target=f, args=(child_conn,))
    p.start()
    print parent_conn.recv()   # prints "[42, None, 'hello']"
    p.join()

[42, None, 'hello']


####synchronization


In [23]:
from multiprocessing import Process, Lock

def f(l, i):
    l.acquire()
    print 'hello world', i
    l.release()

if __name__ == '__main__':
    lock = Lock()

    for num in range(10):
        Process(target=f, args=(lock, num)).start()
        

hello world 0
hello world 1


###Threading

```python
class threading.Thread(group=None, target=None, name=None, args=(), kwargs={})
```

In [ ]:
import threading

def worker(num):
    """thread worker function"""
    print 'Worker: %s' % num
    return

threads = []
for i in range(5):
    t = threading.Thread(target=worker, args=(i,))
    threads.append(t)
    t.start()

####Determining the Current Thread

* Using arguments to identify or name the thread is cumbersome, and unnecessary. 
* Each Thread instance has a name with a default value that can be changed as the thread is created. 
* Naming threads is useful in server processes with multiple service threads handling different operations.


In [24]:

import threading
import time

def worker():
    print threading.currentThread().getName(), 'Starting'
    time.sleep(2)
    print threading.currentThread().getName(), 'Exiting'

def my_service():
    print threading.currentThread().getName(), 'Starting'
    time.sleep(3)
    print threading.currentThread().getName(), 'Exiting'

t = threading.Thread(name='my_service', target=my_service)
w = threading.Thread(name='worker', target=worker)
w2 = threading.Thread(target=worker) # use default name

w.start()
w2.start()
t.start()


worker Starting
Thread-3 Starting
my_service Starting
hello world 2
hello world 3
hello world 5
hello world 4
hello world 6
hello world 7
hello world 9
hello world 8
worker Exiting
Thread-3 Exiting
my_service Exiting


* The __logging__ module supports embedding the thread name in every log message using the formatter code %(threadName)s. 
* Including thread names in log messages makes it easier to trace those messages back to their source.Mb

In [25]:
import logging
import threading
import time

logging.basicConfig(level=logging.DEBUG,
                    format='[%(levelname)s] (%(threadName)-10s) %(message)s',
                    )

def worker():
    logging.debug('Starting')
    time.sleep(2)
    logging.debug('Exiting')

def my_service():
    logging.debug('Starting')
    time.sleep(3)
    logging.debug('Exiting')

t = threading.Thread(name='my_service', target=my_service)
w = threading.Thread(name='worker', target=worker)
w2 = threading.Thread(target=worker) # use default name

w.start()
w2.start()
t.start()


### Subclassing Thread

* At start-up, a Thread does some basic initialization and then calls its __run()__ method, which calls the target function passed to the constructor. 
* To create a subclass of Thread, override __run()__ to do whatever is necessary


In [ ]:
import threading
import logging

logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-10s) %(message)s',
                    )

class MyThread(threading.Thread):

    def run(self):
        logging.debug('running')
        return

for i in range(5):
    t = MyThread()
    t.start()
    

In [ ]:
import threading
import logging

logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-10s) %(message)s',
                    )

class MyThreadWithArgs(threading.Thread):

    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs=None, verbose=None):
        threading.Thread.__init__(self, group=group, target=target, name=name,
                                  verbose=verbose)
        self.args = args
        self.kwargs = kwargs
        return

    def run(self):
        logging.debug('running with %s and %s', self.args, self.kwargs)
        return

for i in range(5):
    t = MyThreadWithArgs(args=(i,), kwargs={'a':'A', 'b':'B'})
    t.start()

###Subprocess 

In [27]:
import subprocess
ls = subprocess.Popen("pwd",
                       shell=False,
                       stdout=subprocess.PIPE,
                       stderr=subprocess.PIPE)
x = ls.stdout.readlines()
print x

['/home/amuneer/workspace/notebook/personal/pace_micro_sep_2015/day3\n']


In [28]:
import subprocess
import sys

HOST="localhost"
# Ports are handled in ~/.ssh/config since we use OpenSSH
COMMAND="uname -a"

ssh = subprocess.Popen(["ssh", "%s" % HOST, COMMAND],
                       shell=False,
                       stdout=subprocess.PIPE,
                       stderr=subprocess.PIPE)
result = ssh.stdout.readlines()
if result == []:
    error = ssh.stderr.readlines()
    print >>sys.stderr, "ERROR: %s" % error
else:
    print result
    

['Linux abdul-work 3.16.0-49-generic #65~14.04.1-Ubuntu SMP Wed Sep 9 10:03:23 UTC 2015 x86_64 x86_64 x86_64 GNU/Linux\n']


 ```python
 subprocess.call(args, *, stdin=None, stdout=None, stderr=None, shell=False)
 ```

    - Run the command described by args. Wait for command to complete, then return the returncode attribute.
    - Do not use stdout=PIPE or stderr=PIPE with this function as that can deadlock based on the child process output volume. 
    - Use Popen with the communicate() method when you need pipes.
    
    
    

 ```python
 subprocess.check_call(args, *, stdin=None, stdout=None, stderr=None, shell=False)
 ```

 - Run command with arguments. Wait for command to complete. If the return code was zero then return, otherwise raise CalledProcessError. The CalledProcessError object will have the return code in the returncode attribute.

 ```python
subprocess.check_output(args, *, stdin=None, stderr=None, shell=False, universal_newlines=False)
```

- Run command with arguments and return its output as a byte string.
- If the return code was non-zero it raises a CalledProcessError. The CalledProcessError object will have the return code in the returncode attribute and any output in the output attribute.